## 1. Web Access

### 1. Modules & Variables

In [5]:
! pip install undetected_chromedriver

# 원격 조종 시에도 구글 로그인 가능
# 참고 링크
# https://github.com/ultrafunkamsterdam/undetected-chromedriver



[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import time
from datetime import datetime

from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.remote.webdriver import By
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
import selenium.webdriver.support.expected_conditions as EC  # noqa

import undetected_chromedriver as uc

from bs4 import BeautifulSoup


In [5]:
dict_gAccounts = {
    'kjaehw0207': 'flapaqj1!%',
    'woghks.study': 'ans_entrance00'
}

account_1 = list(dict_gAccounts.keys())[0]
password_1 = dict_gAccounts[account_1]

account_2 = list(dict_gAccounts.keys())[1]
password_2 = dict_gAccounts[account_2]


### 2. Web Access

In [3]:
# 웹 페이지 접속 - [전체 기록 관리] 페이지로 바로 접근
driver = uc.Chrome()
driver.get(
    'https://myactivity.google.com/product/youtube?hl=ko&utm_medium=web&utm_source=youtube')


In [7]:
# 맞춤 기록 페이지 [로그인] 버튼 클릭
btn_sign_in = WebDriverWait(driver, 3).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="gb"]/div[2]/div[3]/div[1]/a')))
btn_sign_in.click()

# 계정 이메일 입력
input_account = WebDriverWait(driver, 3).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="identifierId"]')))
input_account.send_keys(account_2)

# [다음] 버튼 클릭해 패스워드 입력
btn_next_password = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="identifierNext"]/div/button/span')))
btn_next_password.click()

# 계정 비밀번호 입력
input_password = WebDriverWait(driver, 3).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="password"]/div[1]/div/div[1]/input')))
input_password.send_keys(password_2)

# [다음] 버튼 입력해 로그인 클릭
btn_next_sign_in = WebDriverWait(driver, 3).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="passwordNext"]/div/button')))
btn_next_sign_in.click()


In [11]:
# 연락처 확인을 통한 개인 맞춤 설정 페이지
try:
    # 텍스트를 변수로 정의
    page_personalize = driver.find_element(
        By.XPATH, '//*[@id="yDmH0d"]/c-wiz/div/div/div/div[2]/div[3]/div/div[1]')

    # 변수 정의가 되는 경우. [나중에] 버튼 클릭
    if page_personalize:
        driver.find_element(
            By.XPATH, '//*[@id="yDmH0d"]/c-wiz/div/div/div/div[2]/div[4]/div[1]/button/span').click()

except:
    print('맞춤 페이지 없음. 시청기록 수집을 시작합니다.')


맞춤 페이지 없음. 시청기록 수집을 시작합니다.


In [14]:
driver.refresh()


### 3. Data Collection

#### 1) Functions & Variables

In [15]:
def get_day(weekday=datetime.today().weekday()):
    list = ['월', '화', '수', '목', '금', '토', '일']
    return list[weekday]


def get_date(data_date):
    return datetime.strptime(data_date, '%Y%m%d').strftime('%Y-%m-%d')


In [16]:
# 항목별 Class 이름


# 시청일자 - 목록 content에서 [data-date] 값 추출
nm_class_container = "xDtZAf"  # 각 기록 container / tag = <c-wiz>
nm_class_title = "l8sGWb"  # 영상 제목 & 검색어 / tag = <a>
nm_class_RunningTime = "bI9urf"  # 영상 길이 / tag = <div>
nm_class_WatchTime = "H3Q9vf XTnvW"  # 시청 시간 / tag = <div>
nm_class_channel = "SiEggd"  # 채널명 없는 경우 광고 / tag = <div>

day_today = get_day()
print(day_today)


토


#### 2) DataFrame

In [21]:
import pandas as pd
columns_rec = ['title', 'channel', 'running_time',
               'watch_date', 'watch_time', 'category', 'watched_YN']
df_record = pd.DataFrame(columns=columns_rec)
df_record.head()


,title,channel,running_time,watch_date,watch_time,category,watched_YN


In [22]:
columns_ser = ['searchword', 'search_date', 'search_time']
df_search = pd.DataFrame(columns=columns_ser)
df_search.head()


,searchword,search_date,search_time


#### 3) Web Data Collecting

In [19]:
# 목표 갯수만큼 시청 목록(레코드) 호출

num_target_data = 1000  # int(input("수집할 레코드 수를 100개 단위로 입력해주세요."))
num_press_end = int(num_target_data / 100)

for n in range(num_press_end - 1):
    driver.find_element(By.XPATH, '//body').send_keys(Keys.END)
    time.sleep(2) # 전체 페이지가 뜨도록 Wait

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
contents = soup.find_all('c-wiz', class_='xDtZAf')
print(f'호출된 레코드 수: {len(contents)}')


호출된 레코드 수: 1000


In [23]:
# 레코드 분류 및 카테고리/시청여부 입력

idx_rec = 0  # df_record의 idx
idx_ser = 0  # df_search의 idx
num_ad = 0  # 총 광고 갯수

for idx, content in enumerate(contents):

    if content.find('div', 'SiEggd'):  # 채널명이 있는 경우 - 실제 시청 기록 or 커뮤니티 게시글
        if content.find('div', class_=nm_class_RunningTime): # 영상 길이가 있는 경우 - 실제 시청기록
            print(f"{idx}: Watch Record")

            title = content.find('a', class_=nm_class_title).text
            category = '' #input(f'{title}의 Category는? \t') # 영상 분류 Model 활용을 위해
            channel = content.find('div', class_=nm_class_channel).text 
            running_time = content.find(
                'div', class_=nm_class_RunningTime).text
            watch_date = get_date(content['data-date'])
            watch_time = ' '.join(
                (content.find('div', class_=nm_class_WatchTime).text.split(' '))[:2])
            watched_YN = input(f'{title}을 끝까지 시청하셨나요? \t')

            df_record.loc[idx_rec] = [title, channel,
                                      running_time, watch_date, watch_time, category, watched_YN]
            idx_rec += 1

        else:
            print(f"{idx}: Community Post \t Watch time: {watch_time}")

    else:
        if str(content.find('div', class_='iXL6O')) == '<div class="iXL6O"></div>':
            print(f"{idx}: Search Record")

            title = content.find('a', class_=nm_class_title).text
            search_date = get_date(content['data-date'])
            search_time = ' '.join(
                (content.find('div', class_=nm_class_WatchTime).text.split(' '))[:2])

            df_search.loc[idx_ser] = [title, search_date, search_time]

            idx_ser += 1

        else:
            print(f"{idx}: Ad")

            num_ad += 1


0: Watch Record
1: Ad
2: Watch Record
3: Ad
4: Search Record
5: Ad
6: Watch Record
7: Watch Record
8: Watch Record
9: Watch Record
10: Watch Record
11: Search Record
12: Watch Record
13: Watch Record
14: Ad
15: Search Record
16: Search Record
17: Watch Record
18: Watch Record
19: Ad
20: Watch Record
21: Ad
22: Watch Record
23: Ad
24: Watch Record
25: Watch Record
26: Ad
27: Ad
28: Watch Record
29: Ad
30: Watch Record
31: Watch Record
32: Ad
33: Search Record
34: Search Record
35: Watch Record
36: Search Record
37: Search Record
38: Search Record
39: Watch Record
40: Watch Record
41: Ad
42: Search Record
43: Watch Record
44: Ad
45: Search Record
46: Watch Record
47: Watch Record
48: Ad
49: Ad
50: Watch Record
51: Watch Record
52: Ad
53: Search Record
54: Watch Record
55: Ad
56: Search Record
57: Watch Record
58: Watch Record
59: Ad
60: Watch Record
61: Watch Record
62: Watch Record
63: Search Record
64: Watch Record
65: Search Record
66: Search Record
67: Watch Record
68: Ad
69: Watch R

In [32]:
# 전체 레코드 .json으로 저장하기

idx_rec = 0  # df_record의 idx
idx_ser = 0  # df_search의 idx
num_ad = 0  # 총 광고 갯수

for idx, content in enumerate(contents):

    if content.find('div', 'SiEggd'):  # 채널명이 있는 경우 - 실제 시청 기록 or 커뮤니티 게시글
        if content.find('div', class_=nm_class_RunningTime): # 영상 길이가 있는 경우 - 실제 시청기록
            
            title = content.find('a', class_=nm_class_title).text
            category = '' #input(f'{title}의 Category는? \t') # 영상 분류 Model 활용을 위해
            channel = content.find('div', class_=nm_class_channel).text 
            running_time = content.find(
                'div', class_=nm_class_RunningTime).text
            watch_date = get_date(content['data-date'])
            watch_time = ' '.join(
                (content.find('div', class_=nm_class_WatchTime).text.split(' '))[:2])

            print(f"{idx}: Watch Record\t\t Running_time: {running_time}\t watch_time: {watch_time}")

        else:
            print(f"{idx}: Community Post\t\t Watch time: {watch_time}")

    else:
        if str(content.find('div', class_='iXL6O')) == '<div class="iXL6O"></div>':

            title = content.find('a', class_=nm_class_title).text
            search_date = get_date(content['data-date'])
            search_time = ' '.join(
                (content.find('div', class_=nm_class_WatchTime).text.split(' '))[:2])
            
            print(f"{idx}: Search Record\t\t Search time = {search_time}")

        else:
            print(f"{idx}: Ad\t\t\t watch_time: {watch_time}")


0: Watch Record		 Running_time: 24:35	 watch_time: 오후 1:18
1: Ad			 watch_time: 오후 1:18
2: Watch Record		 Running_time: 14:55	 watch_time: 오전 11:27
3: Ad			 watch_time: 오전 11:27
4: Search Record		 Search time = 오전 11:12
5: Ad			 watch_time: 오전 11:27
6: Watch Record		 Running_time: 12:58	 watch_time: 오전 8:36
7: Watch Record		 Running_time: 7:55	 watch_time: 오전 8:33
8: Watch Record		 Running_time: 3:28	 watch_time: 오후 11:50
9: Watch Record		 Running_time: 11:18	 watch_time: 오후 11:46
10: Watch Record		 Running_time: 0:59	 watch_time: 오후 11:45
11: Search Record		 Search time = 오후 11:45
12: Watch Record		 Running_time: 7:58	 watch_time: 오후 11:42
13: Watch Record		 Running_time: 8:19	 watch_time: 오후 11:39
14: Ad			 watch_time: 오후 11:39
15: Search Record		 Search time = 오후 11:38
16: Search Record		 Search time = 오후 11:38
17: Watch Record		 Running_time: 19:21	 watch_time: 오후 11:27
18: Watch Record		 Running_time: 8:22	 watch_time: 오후 11:27
19: Ad			 watch_time: 오후 11:27
20: Watch Record		 Run

In [24]:
df_record.to_csv('230211_watch_record.csv', encoding='utf-8')
df_record


,title,channel,running_time,watch_date,watch_time,category,watched_YN
0,[한눈에 이슈] '요즘 누가 검색을 하니?'...구글의 충격적인 미래 현실로 / K...,KBS News,24:35,2023-02-11,오후 1:18,,0
1,[아고라스쿨] 의료기기 인허가 프로세스에 대해서 알아보자!,보건산업혁신창업센터 KBIC,14:55,2023-02-11,오전 11:27,,0
2,챗 GPT 열풍! 인공지능(AI) 관련주 어떻게 될까,황현희 - 조사하면 돈 나와,12:58,2023-02-11,오전 8:36,,1
3,죄송합니다...,도레,7:55,2023-02-11,오전 8:33,,0
4,현직자가 말하는 요즘 신입사원 나이?,캐치TV,3:28,2023-02-10,오후 11:50,,0
...,...,...,...,...,...,...,...
413,"'SKY' 수시 합격생 1/3 등록 포기...""의대 지원 강화"" / YTN",YTN,2:20,2022-12-25,오후 10:14,,1
414,How to Improve Your Decision-Making,The Art of Improvement,8:20,2022-12-25,오후 10:14,,0
415,성탄절 기념 무료배포 (영구소장),방앗간 비둘기,1:01,2022-12-25,오후 2:23,,1
416,크리스마스엔 핫초코 먹으면서 페이커 영상 보는 게 국룰이죠! [Faker Strea...,T1 Faker,7:05,2022-12-25,오후 2:17,,1


In [25]:
df_search.to_csv('230211_search_record.csv', encoding='utf-8')
df_search


,searchword,search_date,search_time
0,의료기기 엔지니어링,2023-02-11,오전 11:12
1,데이터 엔지니어,2023-02-10,오후 11:45
2,면접왕 이형,2023-02-10,오후 11:38
3,사망여우,2023-02-10,오후 11:38
4,페이지,2023-02-10,오후 7:13
...,...,...,...
163,Rtx3080ti vs gtx1060,2022-12-27,오후 1:36
164,Rtx 4090,2022-12-27,오후 1:36
165,랄로,2022-12-27,오후 12:53
166,파카,2022-12-27,오후 12:53


In [26]:
print(
    f"입력한 시청기록 레코드는 {num_target_data}개, \n실제 시청한 영상은 {len(df_record)}개, \n광고는 {num_ad}개입니다.")


입력한 시청기록 레코드는 1000개, 
실제 시청한 영상은 418개, 
광고는 404개입니다.


In [ ]:
# ERROR - 커뮤니티 글 확인 기록
# 채널은 있는데, 영상이 없음
"""
80: NO ads
지금 이 순간 대생성AI 버블 시대가 열렸습니다. 닷컴버블, 스마트폰 혁명, 블록체인 열풍을 놓치신 개발자분들 이번 기회는 꼭 잡으시길 바랍니다!
🎬영상보기: https://youtu.be/4xU0mGr4g...
---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
Cell In[33], line 9
      7     print(f'{idx}: NO ads')
      8     print(content.find('a', class_=nm_class_title).text)
----> 9     print(content.find('div', class_=nm_class_RunningTime).text)
     10     print(' '.join((content.find('div', class_=nm_class_WatchTime).text.split(' '))[:2]))
     12 else:

AttributeError: 'NoneType' object has no attribute 'text'
"""


### Spare codes

In [ ]:
btn_acccount_info = driver.find_element(
    By.XPATH, '//*[@id="gb"]/div[2]/div[3]/div[1]/div[2]/div/a')

btn_sign_out = ''
# html document, iframe 태그 안에 들어있어 확인이 어려움.
# 여러 계정을 같이 분석할 경우 driver 초기화 방식으로 진행하자.


In [72]:
driver.close()


In [ ]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

contents = soup.find_all('c-wiz', class_='xDtZAf')
for idx, content in enumerate(contents):
    if content.find('div', 'SiEggd'):
        if content.find('div', class_=nm_class_RunningTime):
            print(f"{idx}: Watch Record")
            print(f"Watch Date: {get_date(content['data-date'])}")
            print(f"Title: {content.find('a', class_=nm_class_title).text}")
            print(
                f"Running Time: {content.find('div', class_=nm_class_RunningTime).text}")
            print(
                f"Watch Time: {' '.join((content.find('div', class_=nm_class_WatchTime).text.split(' '))[:2])}")
        else:
            print(f"{idx}: Community Post")

    else:
        if str(content.find('div', class_='iXL6O')) == '<div class="iXL6O"></div>':
            print(f"{idx}: Search")
            print(f"Search Date: {get_date(content['data-date'])}")
            print(
                f"Search Word: {content.find('a', class_=nm_class_title).text}")
            print(
                f"Search Time: {' '.join((content.find('div', class_=nm_class_WatchTime).text.split(' '))[:2])}")
        else:
            print(f"{idx}: Ad")

    print(' ')


0: Ad
 
1: Ad
 
2: Watch Record
Watch Date: 2023-02-04
Title: 초가스 키우기 | 디알엑스 vs. 한화생명 게임1 하이라이트 | 02.04 | 2023 LCK 스프링 스플릿
Running Time: 12:33
Watch Time: 오후 8:19
 
3: Ad
 
4: Ad
 
5: Watch Record
Watch Date: 2023-02-04
Title: 신궁 | 농심 vs. T1 게임1 하이라이트 | 02.04 | 2023 LCK 스프링 스플릿
Running Time: 9:38
Watch Time: 오후 8:10
 
6: Ad
 
7: Watch Record
Watch Date: 2023-02-04
Title: [스티브 잡스] A급 인재와 B급, C급 직원의 차이 (한영 자막)
Running Time: 7:27
Watch Time: 오후 8:02
 
8: Search
Search Date: 2023-02-04
Search Word: 파카
Search Time: 오전 2:02
 
9: Watch Record
Watch Date: 2023-02-04
Title: Crimson Raptor Champion Spotlight | Parody - League of Legends
Running Time: 9:45
Watch Time: 오전 1:58
 
10: Ad
 
11: Watch Record
Watch Date: 2023-02-04
Title: 젠지 vs. 리브 샌박 | 매치26 하이라이트 | 02.03 | 2023 LCK 스프링 스플릿
Running Time: 45:49
Watch Time: 오전 12:54
 
12: Watch Record
Watch Date: 2023-02-04
Title: KT vs. 광동 | 매치25 하이라이트 | 02.03 | 2023 LCK 스프링 스플릿
Running Time: 26:07
Watch Time: 오전 12:53
 
13: Ad
 
14: Ad
 
15: Watch Reco

### Troubleshooting

아이디 / 비밀번호 입력 없이 로그인을 직접 한다면?
- 보안 문제
- 로그인 한 후 driver 초기화 문제
    - 1) driver 재시작
    - 2) 